In [ ]:
import pandas as pd

# Load both datasets
bert_gq1_lq_scores = pd.read_csv('/home/qiyu/Dev/ziqing/T5/train/eval_newcsQGSE/T5_newcs_once/BERT30_gq1_lq.csv')
human_individual = pd.read_csv('/home/qiyu/Dev/ziqing/T5/human_new_individual.csv')
bert_gq2_lq_scores = pd.read_csv('/home/qiyu/Dev/ziqing/T5/train/eval_newcsQGSE/T5_newcs_once/BERT30_gq2_lq.csv')


def update_bert_score1(row):
    if row['Question Type'] == 'rel':
        
        index = row['ID'] - 1  
        if 0 <= index < len(bert_gq1_lq_scores):
            return bert_gq1_lq_scores.iloc[index]['F1 Score']
    return row['BERTScore']

def update_bert_score2(row):
    if row['Question Type'] == 'non-rel':
       
        index = row['ID'] - 1  
        if 0 <= index < len(bert_gq2_lq_scores):
            return bert_gq2_lq_scores.iloc[index]['F1 Score']
    return row['BERTScore']


human_individual['BERTScore'] = human_individual.apply(update_bert_score1, axis=1)
human_individual['BERTScore'] = human_individual.apply(update_bert_score2, axis=1)


updated_csv_filename = 'humanBERT_new_individual.csv'
human_individual.to_csv(updated_csv_filename, index=False)

# Print the path to the saved file
print(f"Updated CSV file has been saved to {updated_csv_filename}")


In [ ]:
import pandas as pd
import json


with open('/home/qiyu/Dev/ziqing/T5/train/eval_newcsQGSE/T5_newcs_once/TOPIC_eval/relate_labeltopic.json', 'r') as file:
    json_data = json.load(file)


relatedness_df = pd.DataFrame(json_data)
relatedness_df.set_index('sentence_index', inplace=True)


human_individual = pd.read_csv('/home/qiyu/Dev/ziqing/T5/T5_human_new_individual.csv')


def update_relatedness_score(row):
    if row['Question Type'] == 'rel':
        
        index = row['ID'] - 1
        if index in relatedness_df.index:
            return relatedness_df.loc[index, 'relatedness_score']
    return row['Relatedness Score(jaccard)']


human_individual['Relatedness Score(jaccard)'] = human_individual.apply(update_relatedness_score, axis=1)


updated_csv_filename = 'T5_human_new_individual.csv'
human_individual.to_csv(updated_csv_filename, index=False)


In [ ]:
import pandas as pd
import json

with open('/home/qiyu/Dev/ziqing/T5/train/eval_newcsQGSE/T5_newcs_once/TOPIC_w2v/relate_othertopic.json', 'r') as file:
    json_data = json.load(file)


relatedness_df = pd.DataFrame(json_data)
relatedness_df.set_index('sentence_index', inplace=True)


human_individual = pd.read_csv('/home/qiyu/Dev/ziqing/T5/T5_human_new_individual.csv')


def update_relatedness_score(row):
    if row['Question Type'] == 'non-rel':

        index = row['ID'] - 1
        if index in relatedness_df.index:
            return relatedness_df.loc[index, 'relatedness_score']
    return row['Relatedness Score(w2v)']


human_individual['Relatedness Score(w2v)'] = human_individual.apply(update_relatedness_score, axis=1)


human_individual = human_individual.round(3)


updated_csv_filename = 'T5_human_new_individual.csv'
human_individual.to_csv(updated_csv_filename, index=False)



In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error


df = pd.read_csv('T5_human_new_individual.csv')

df['Human Label'] = pd.to_numeric(df['Human Label'], errors='coerce')
df['Relatedness Score'] = pd.to_numeric(df['Relatedness Score(w2v)'], errors='coerce')

df.dropna(subset=['Human Label', 'Relatedness Score(w2v)'], inplace=True)

mae_R = mean_absolute_error(df['Human Label'], df['Relatedness Score(w2v)'])

print(f"The Mean Absolute Error between Human Label and Relatedness Score(w2v) is: {mae_R}")


In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error


df = pd.read_csv('T5_human_new_individual.csv')


df['Human Label'] = pd.to_numeric(df['Human Label'], errors='coerce')
df['Relatedness Score'] = pd.to_numeric(df['Relatedness Score(jaccard)'], errors='coerce')


df.dropna(subset=['Human Label', 'Relatedness Score(jaccard)'], inplace=True)


mae_R_jaccard = mean_absolute_error(df['Human Label'], df['Relatedness Score(jaccard)'])


print(f"The Mean Absolute Error between Human Label and Relatedness Score(jaccard) is: {mae_R_jaccard}")


In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error


df = pd.read_csv('T5_human_new_individual.csv')


df['Human Label'] = pd.to_numeric(df['Human Label'], errors='coerce')
df['BERTScore'] = pd.to_numeric(df['BERTScore'], errors='coerce')

df.dropna(subset=['Human Label', 'BERTScore'], inplace=True)

mae_B = mean_absolute_error(df['Human Label'], df['BERTScore'])

print(f"The Mean Absolute Error between Human Label and BERTScore is: {mae_B}")


In [ ]:
# Format the results into a string for clear presentation
result_content = f"""
Human Evaluation:
-------------------------------------------------
The Mean Absolute Error between Human Label and BERTScore is: {mae_B}

The Mean Absolute Error between Human Label and Relatedness Score(jaccard) is: {mae_R_jaccard} 

The Mean Absolute Error between Human Label and Relatedness Score(w2v) is: {mae_R} 

-------------------------------------------------
"""

# Print the results to check output
print(result_content)

# Write the content to a txt file
file_path = "/home/qiyu/Dev/ziqing/T5/train/eval_newcsQGSE/T5_newcs_once/human_new_mae.txt"
with open(file_path, "w") as file:
    file.write(result_content)

# Displaying the file path for the user to download
file_path

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/home/qiyu/Dev/ziqing/T5/human_individual.csv')

# Rename columns
df = df.rename(columns={
    'Relatedness Score': 'Relatedness Score(jaccard)'
})

# Assuming 'Relatedness Score(w2v)' needs to be added as a new column with default values
# Here I assume a default value of NaN; adjust as needed
df['Relatedness Score(w2v)'] = pd.NA

# Reorder columns
df = df[['ID', 'Question Type', 'Human Label', 'BERTScore', 'Relatedness Score(jaccard)', 'Relatedness Score(w2v)']]

# Display the first few rows of the DataFrame to check changes
print(df.head())

# Save the modified DataFrame back to a CSV file
df.to_csv('human_new_individual.csv', index=False)
